In [1]:
import matplotlib

matplotlib.use('TkAgg')
from Recordings import Recording, AEPRecording, AEPFeedbackRecording, Recordings
import pandas as pd
import numpy as np
import os
import mne

The following cell loads all the raw data from the XDF files across different sessions and subjects. Data validation warnings are all manually checked and are fixed at the point the data is loaded.

In [2]:
# Load the data
basepath = './Recordings'
recordings = Recordings(basepath)

Reading recording: ./Recordings/1/9/1/9_aep_2024-06-10_14-32-42_1.xdf
Reading recording: ./Recordings/1/9/1/9_aep_feedback_2024-06-10_14-40-46_1.xdf
Reading recording: ./Recordings/1/9/10/9_resting_closed_2024-06-10_14-29-49_10.xdf
Reading recording: ./Recordings/1/9/10/9_resting_open_2024-06-10_14-26-10_10.xdf
Reading recording: ./Recordings/1/9/2/9_aep_feedback_2024-06-10_14-56-51_2.xdf
WARNING, data validation failed. Recording 9_aep_feedback_2024-06-10_14-56-51_2.xdf does not match its metadata.
Reading recording: ./Recordings/1/9/2/9_aep_2024-06-10_14-50-06_2.xdf
WARNING, data validation failed. Recording 9_aep_2024-06-10_14-50-06_2.xdf does not match its metadata.
Reading recording: ./Recordings/1/7/1/7_aep_2024-03-28_14-34-11_1.xdf
Reading recording: ./Recordings/1/7/1/7_aep_feedback_2024-03-28_14-41-11_1.xdf
Reading recording: ./Recordings/1/7/10/7_resting_closed_2024-05-23_16-27-08_10.xdf
Reading recording: ./Recordings/1/7/10/7_resting_open_2024-05-23_16-22-06_10.xdf
Reading 

First we export the raw data in CSV files. All the XDF files are simply converted to tables, where each column is a channel from Unicorn Hybrid Black EEG device, with an additional column for event markers.

In [3]:
# We first create Pandas table for each recording file
output_tables = []
for recording in recordings.recordings:
    data = {
        "Time": recording.eeg_time,
        "EEG_Fz": recording.eeg_data[:, 0],
        "EEG_C3": recording.eeg_data[:, 1],
        "EEG_Cz": recording.eeg_data[:, 2],
        "EEG_C4": recording.eeg_data[:, 3],
        "EEG_Pz": recording.eeg_data[:, 4],
        "EEG_PO7": recording.eeg_data[:, 5],
        "EEG_Oz": recording.eeg_data[:, 6],
        "EEG_PO8": recording.eeg_data[:, 7],
        "Accelerometer_X": recording.accelerometer_data[:, 0],
        "Accelerometer_Y": recording.accelerometer_data[:, 1],
        "Accelerometer_Z": recording.accelerometer_data[:, 2],
        "Gyroscope_X": recording.gyroscope_data[:, 0],
        "Gyroscope_Y": recording.gyroscope_data[:, 1],
        "Gyroscope_Z": recording.gyroscope_data[:, 2],
        "Battery_Level": recording.battery_level_data,
        "Counter": recording.counter_data,
        "Validation": recording.validation_indicator_data,
    }
    df = pd.DataFrame(data=data)
    indices_array = [(np.abs(df['Time'] - value)).argmin() for value in recording.marker_time]
    # Avoid collision of events
    for i, val in enumerate(indices_array):
        if i > 0 and indices_array[i - 1] >= val:
            indices_array[i] = indices_array[i - 1] + 1
    for index, event in zip(indices_array, recording.marker_data):
        df.at[index, 'Event'] = event
    output_tables.append(df)

In [4]:
# Temporary: Check if output table event counts are the same (trial-end vs trial-begin)
for i, table in enumerate(output_tables):
    print(f"Recording {i + 1}:")
    if 'trial-begin' not in table['Event'].value_counts():
        print("No trial-begin event")
    else:
        print(table['Event'].value_counts()['trial-end'] - table['Event'].value_counts()['trial-begin'])

Recording 1:
0
Recording 2:
0
Recording 3:
No trial-begin event
Recording 4:
No trial-begin event
Recording 5:
0
Recording 6:
0
Recording 7:
0
Recording 8:
0
Recording 9:
No trial-begin event
Recording 10:
No trial-begin event
Recording 11:
0
Recording 12:
0
Recording 13:
0
Recording 14:
0
Recording 15:
No trial-begin event
Recording 16:
No trial-begin event
Recording 17:
0
Recording 18:
0
Recording 19:
0
Recording 20:
0
Recording 21:
No trial-begin event
Recording 22:
No trial-begin event
Recording 23:
0
Recording 24:
0
Recording 25:
0
Recording 26:
0
Recording 27:
-1
Recording 28:
No trial-begin event
Recording 29:
No trial-begin event
Recording 30:
0
Recording 31:
0
Recording 32:
0
Recording 33:
0
Recording 34:
0
Recording 35:
0
Recording 36:
0
Recording 37:
0
Recording 38:
No trial-begin event
Recording 39:
No trial-begin event
Recording 40:
0
Recording 41:
0
Recording 42:
0
Recording 43:
0
Recording 44:
No trial-begin event
Recording 45:
No trial-begin event
Recording 46:
0
Record

## Data Description

Output folder structure:
- Raw_Data/
    - AEP/
    - AEP_Feedback/
    - Resting_Open/
    - Resting_Closed/
- Filtered_Data/
    - AEP/
    - AEP_Feedback/
    - Resting_Open/
    - Resting_Closed/
- Epoched_Data/
    - ...
- Selected_Epochs/
    - ...


## Raw Data Description

A folder named "Raw_Data" contains unsegmented, unfiltered and unprocessed data in CSV format. Each folder in "Raw_Data" that corresponds to one of the four experiment types will contain recordings from each subject.

Recording files inside these folders are be named as such: `{Subject ID}_{Experiment Type}_{Session ID}.csv`. 

For example, resting state recording (with eyes open) from the first subject during the second session will have the name: `1_resting-open_2.csv`.

Subjects are numbered from 1 through 10 and possible experiment types are aep/aep-feedback/resting-open/resting-closed. Session IDs are either 1 or 2 for 'aep/aep-feedback' experiments, while all resting state experiments have a single session.

Some recordings will have an additional '_UNFINISHED' suffix at the end, which means that the recording was interrupted during during that session (for example, due to EEG device battery getting too low). All '_UNFINISHED' recordings were re-run from start to finish, which means there are at least two full sessions for 'aep/aep-feedback' experiments. However, epochs from unfinished recordings do not have any defects and can be freely used in the subsequent steps of processing.  

In [5]:
def export_recordings(base_folder='Raw_Data', table_transform_fn=None):
    for i, recording in enumerate(recordings.recordings):
        # Find out if the current recording is unfinished
        is_current_unfinished = False
        if recording.experiment_id == 'aep' or recording.experiment_id == 'aep_feedback':
            similar_recordings = [r for r in recordings.recordings if
                                  r.subject_id == recording.subject_id and r.experiment_id == recording.experiment_id and r.session == recording.session]
            if len(similar_recordings) > 1:  # There will always be at least 1 similar recording (self)
                similar_recordings = sorted(similar_recordings, key=lambda
                    x: x.recording_name)  # This will sort by name which also sorts by time recorded in this case
                # Checks if current recording was recorded before the other (similar) one, in which case the current must be labeled as unfinished
                if recording.recording_name == similar_recordings[0].recording_name:
                    print(
                        f'Recording named {recording.recording_name} is unfinished, while {similar_recordings[1].recording_name} is not.')
                    is_current_unfinished = True
        # Create an output directory and name the output file
        if recording.experiment_id == 'aep':
            subfolder = "AEP"
        elif recording.experiment_id == 'aep_feedback':
            subfolder = "AEP_Feedback"
        elif recording.experiment_id == 'resting_open':
            subfolder = "Resting_Open"
        else:
            subfolder = "Resting_Closed"
        output_dir = f'./{base_folder}/{subfolder}/'
        output_filename = f"{recording.subject_id}_{recording.experiment_id.replace('_', '-')}_{recording.session}"
        if is_current_unfinished:
            output_filename += '_UNFINISHED'
        output_path = f'{output_dir}{output_filename}.csv'
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        # Transform the output table if the transform function is passed
        if table_transform_fn is not None:
            output_tables[i] = table_transform_fn(output_tables[i])
        # Export the recording file
        output_tables[i].to_csv(output_path)

In [6]:
export_recordings('Raw_Data', None)

Recording named 10_aep_2024-07-07_12-58-52_1.xdf is unfinished, while 10_aep_2024-07-07_13-30-26_1.xdf is not.
Recording named 2_aep_feedback_2024-03-18_15-42-22_1.xdf is unfinished, while 2_aep_feedback_2024-03-27_13-00-43_1.xdf is not.


## File Structure

Each CSV file recording has 19 columns. Each column corresponds to an output channel acquired from the Unicorn Hybrid Black EEG device. There are different types of channels in the raw recording files: 1 time channel, 8 EEG channels, 3 accelerometer channels, 3 gyroscope channels, 1 counter channel, 1 validation channel, and 1 event channel. 

Specifically, all the 19 channels are: Time, EEG_Fz, EEG_C3, EEG_Cz, EEG_C4, EEG_Pz, EEG_PO7, EEG_Oz, EEG_PO8, Accelerometer_X, Accelerometer_Y, Accelerometer_Z, Gyroscope_X, Gyroscope_Y, Gyroscope_Z, Battery_Level, Counter, Validation, Event.

1. *Time* column indicates number of seconds passed since the start of the session.
2. *EEG_\** columns indicate unfiltered voltage values (in microvolts) for 8 EEG channels specified in Unicord Hybrid Black EEG device manual. For each EEG channel an electrode position is indicated (e.g. Cz) according to international 10-20 system.
3. *Accelerometer_\** columns indicate acceleration (±8 g) of the Unicorn Hybrid Black EEG device in X/Y/Z directions.
4. *Gyroscope_\** columns indicate the angular rotation (±1000 °/s) of Unicorn Hybrid Black EEG device in X/Y/Z directions.
5. *Battery_Level* column value ranges from 0 to 100 and indicates the remaining battery level.
6. *Counter* column tracks the sample order in which the values were received from Unicorn Hybrid Black to the host PC.
7. *Validation* column is a validation indicator for the samples received from Unicorn Hybrid Black device.
8. *Event* column indicates if any event was registered during the recording. Event descriptions for different experiment types can be found below.

More information about channels can be found in [Unicorn Hybrid Black User Manual](https://github.com/unicorn-bi/Unicorn-Suite-Hybrid-Black-User-Manual).

The data was streamed using Unicorn LSL interface, which by itself outputs 17 raw channels (except "Time" and "Events" channels). Given channels were exported into CSV files without any modification. Event markers were streamed (also using LSL protocol) by NeuroPype™ Experiment Recorder (ER) software. Both of these streams were exported in XDF file format and then automatically synced using PyXDF library. As a result, "Time" and "Events" columns were added to the CSV files.

## Filtered Data

The filtered data is in "Filtered_Data" folder. The filtered data has the same CSV file format as the raw data. Only EEG channels are modified, other columns remain the same.

Filtering of the EEG channels was done using the 0.5-60 Hz bandpass filter and 50 Hz notch filter. Both filters were designed with the MNE library as one-pass, zero-phase, non-causal filters using the windowed time-domain (firwin) method with a Hamming window.

In [7]:
def filtered_data_transform_fn(dataframe):
    info = mne.create_info(ch_names=['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8'], ch_types=['eeg'] * 8, sfreq=250)
    np_arr = np.transpose(
        dataframe[['EEG_Fz', 'EEG_C3', 'EEG_Cz', 'EEG_C4', 'EEG_Pz', 'EEG_PO7', 'EEG_Oz', 'EEG_PO8']].to_numpy())
    raw = mne.io.RawArray(np_arr, info)
    raw.notch_filter(freqs=[47, 50, 53])
    raw.filter(0.5, 60)
    data = np.transpose(raw.get_data())
    return dataframe.assign(
        EEG_Fz=data[:, 0],
        EEG_C3=data[:, 1],
        EEG_Cz=data[:, 2],
        EEG_C4=data[:, 3],
        EEG_Pz=data[:, 4],
        EEG_PO7=data[:, 5],
        EEG_Oz=data[:, 6],
        EEG_PO8=data[:, 7]
    )

# Also modifies the output_tables
export_recordings('Filtered_Data', filtered_data_transform_fn)

Creating RawArray with float64 data, n_channels=8, n_times=112730
    Range : 0 ... 112729 =      0.000 ...   450.916 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 60.00 Hz
- Up

## Epoched Data

The raw data for each subject were segmented using the event markers, where the all the data that was not involved in the trials (e.g. resting sections) were removed. For the "AEP" and "AEP_Feedback" experiments, segmented data epochs include the period from -0.2s before stimulus. For the "Resting_Open" and "Resting_Closed" experiments, starting and ending points are cropped to the nearest event markers. The resulting epochs are stored in the "Epoched_Data" folder. No additional processing was done on the data except for filtering process described above.

Each epoch is stored in a separate CSV file while the multiple sessions are merged. Additionally, a suffix is added to the file name to indicate the stimulus type, which is either "standard" or "oddball". The naming convention for the files is as follows: `{Subject ID}_{Experiment Type}_{Epoch ID}_{Stimulus Type}.csv`.

In [8]:
epoch_counters = {}
for r, recording in enumerate(recordings.recordings):
    epoch_counters[recording.subject_id] = epoch_counters.get(recording.subject_id, 1)
    # Read events of the current recording as a list of tuples: (event_index, event_label)
    event_series = output_tables[r]['Event'].astype(str)
    # Skip the NaN values
    event_indices = [i for i, event in enumerate(event_series) if event != 'nan']
    events = [event for i, event in enumerate(event_series) if event != 'nan']
    if recording.experiment_id == 'aep':
        assert abs(events.count('trial-begin') - events.count('trial-end')) < 2 
        for i, event in enumerate(events):
            if event == 'trial-begin':
                is_end_of_block = False
                try:
                    assert events[i + 1] in ['standard', 'oddball'], i
                    # Search for 'trial-end', watch out for errors
                    trial_end_index = None
                    for j, event2 in enumerate(events[i + 1:]):
                        if 'error' in event2:
                            break
                        if 'response-received-enter' in event2:
                            is_end_of_block = True
                        if event2 == 'trial-end':
                            trial_end_index = i + 1 + j
                            break
                    assert trial_end_index is not None, i
                    assert events[trial_end_index] == 'trial-end', i
                    assert event_indices[i] != event_indices[trial_end_index] - 2, i
                except AssertionError as e:
                    print(f"WARNING, data validation failed. Skipping trial...", e)
                    continue
                # Create a new epoch
                if is_end_of_block:
                    end_index = event_indices[i] + int(1.3 * 250) # Force finish 1.3s after the stimulus
                else:
                    end_index = event_indices[trial_end_index]
                pre_epoch_samples = int(0.2 * 250)  # Calculate 0.2s before the stimulus in samples
                epoch = output_tables[r].iloc[event_indices[i] - pre_epoch_samples:end_index + 1].copy()
                if is_end_of_block:
                    epoch.loc[end_index, 'Event'] = 'trial-end'  # Add the trial-end event if it was missing
                # Clear any events before the start of the stimulus
                epoch.loc[epoch.index < event_indices[i], 'Event'] = np.nan
                # Correct the time and samples for the epoch
                epoch.loc[:, 'Time'] = epoch['Time'] - epoch.loc[event_indices[i], 'Time']
                epoch.reset_index(drop=True, inplace=True)
                # Export the epoch
                output_dir = f'./Epoched_Data/AEP/'
                output_filename = f"{recording.subject_id}_{recording.experiment_id.replace('_', '-')}_{epoch_counters[recording.subject_id]}_{'standard' if events[i + 1] == 'standard' else 'oddball'}"
                output_path = f'{output_dir}{output_filename}.csv'
                os.makedirs(output_dir, exist_ok=True)
                epoch.to_csv(output_path)
                epoch_counters[recording.subject_id] += 1

WARNING, data validation failed. Skipping trial... 311
WARNING, data validation failed. Skipping trial... 314
WARNING, data validation failed. Skipping trial... 317
WARNING, data validation failed. Skipping trial... 320
WARNING, data validation failed. Skipping trial... 323
WARNING, data validation failed. Skipping trial... 326
WARNING, data validation failed. Skipping trial... 329
WARNING, data validation failed. Skipping trial... 332
WARNING, data validation failed. Skipping trial... 336
WARNING, data validation failed. Skipping trial... 339
WARNING, data validation failed. Skipping trial... 342
WARNING, data validation failed. Skipping trial... 345
WARNING, data validation failed. Skipping trial... 348
WARNING, data validation failed. Skipping trial... 351
WARNING, data validation failed. Skipping trial... 354
WARNING, data validation failed. Skipping trial... 359
WARNING, data validation failed. Skipping trial... 362
WARNING, data validation failed. Skipping trial... 365
WARNING, d

In [16]:
epoch_counters = {}
for r, recording in enumerate(recordings.recordings):
    epoch_counters[recording.subject_id] = epoch_counters.get(recording.subject_id, 1)
    # Read events of the current recording as a list of tuples: (event_index, event_label)
    event_series = output_tables[r]['Event'].astype(str)
    # Skip the NaN values
    event_indices = [i for i, event in enumerate(event_series) if event != 'nan']
    events = [event for i, event in enumerate(event_series) if event != 'nan']
    if recording.experiment_id == 'aep_feedback':
        for i, event in enumerate(events):
            is_end_of_block = False
            if event == 'trial-begin':
                try:
                    assert events[i + 1] in ['standard', 'oddball'], i
                    assert 'response-received' in events[i + 2] or 'was-missed' in events[i + 2], i
                    # Search for 'trial-end', watch out for errors
                    trial_end_index = None
                    for j, event2 in enumerate(events[i + 3:]):
                        if 'error' in event2:
                            break
                        # if 'rt' in event2 and 
                        if event2 == 'trial-end':
                            trial_end_index = i + 3 + j
                            # Check that trial-end is directly after 'rt-*' event, if not, it is the end of the block
                            is_end_of_block = event_indices[trial_end_index - 1] != event_indices[trial_end_index] - 1
                            break
                    assert trial_end_index is not None, i
                    assert events[trial_end_index] == 'trial-end', i
                    assert 'rt' in events[trial_end_index - 1], i
                    assert event_indices[i] != event_indices[trial_end_index] - 4, i
                except AssertionError as e:
                    print(f"WARNING, data validation failed. Skipping trial...", e)
                    continue
                # Create a new epoch
                if is_end_of_block:
                    end_index = event_indices[i] + int(1.2 * 250) # Force finish 1.2s after the stimulus
                else:
                    end_index = event_indices[trial_end_index]
                pre_epoch_samples = int(0.2 * 250)  # Calculate 0.2s before the stimulus in samples
                epoch = output_tables[r].iloc[event_indices[i] - pre_epoch_samples:end_index + 1].copy()
                if is_end_of_block:
                    epoch.loc[end_index, 'Event'] = 'trial-end'  # Add the trial-end event if it was missing
                # Clear any events before the start of the stimulus
                epoch.loc[epoch.index < event_indices[i], 'Event'] = np.nan
                # Correct the time and samples for the epoch
                epoch.loc[:, 'Time'] = epoch['Time'] - epoch.loc[event_indices[i], 'Time']
                epoch.reset_index(drop=True, inplace=True)
                # Export the epoch
                output_dir = f'./Epoched_Data/AEP_Feedback/'
                output_filename = f"{recording.subject_id}_{recording.experiment_id.replace('_', '-')}_{epoch_counters[recording.subject_id]}_{'standard' if events[i + 1] == 'standard' else 'oddball'}"
                output_path = f'{output_dir}{output_filename}.csv'
                os.makedirs(output_dir, exist_ok=True)
                epoch.to_csv(output_path)
                epoch_counters[recording.subject_id] += 1

WARNING, data validation failed. Skipping trial... 758
WARNING, data validation failed. Skipping trial... 766
WARNING, data validation failed. Skipping trial... 771
WARNING, data validation failed. Skipping trial... 778
WARNING, data validation failed. Skipping trial... 783
WARNING, data validation failed. Skipping trial... 788
WARNING, data validation failed. Skipping trial... 793
WARNING, data validation failed. Skipping trial... 798
WARNING, data validation failed. Skipping trial... 803
WARNING, data validation failed. Skipping trial... 808
WARNING, data validation failed. Skipping trial... 813
WARNING, data validation failed. Skipping trial... 819
WARNING, data validation failed. Skipping trial... 824
WARNING, data validation failed. Skipping trial... 829
WARNING, data validation failed. Skipping trial... 834
WARNING, data validation failed. Skipping trial... 839
WARNING, data validation failed. Skipping trial... 844
WARNING, data validation failed. Skipping trial... 849
WARNING, d

In [25]:
# Crop start and end points for resting state recordings
for r, recording in enumerate(recordings.recordings):
    if recording.experiment_id == 'resting_open' or recording.experiment_id == 'resting_closed':
        # Read events of the current recording as a list of tuples: (event_index, event_label)
        event_series = output_tables[r]['Event'].astype(str)
        # Skip the NaN values
        event_indices = [i for i, event in enumerate(event_series) if event != 'nan']
        events = [event for i, event in enumerate(event_series) if event != 'nan']
        for i, event in enumerate(events):
            if 'resting-state' in event and 'begin' in event:
                try:
                    assert 'resting-state' in events[i + 1], i
                except AssertionError as e:
                    # print(events[i + 1])
                    # print(recording.subject_id, recording.experiment_id)
                    print(f"WARNING, data validation failed. Skipping trial...", e)
                    continue
                # Create a new epoch
                epoch = output_tables[r].iloc[event_indices[i]:event_indices[i + 1] + 1]
                # Correct the time and samples for the epoch
                epoch.loc[:, 'Time'] = epoch['Time'] - epoch.loc[event_indices[i], 'Time']
                epoch.reset_index(drop=True, inplace=True)
                # Export the epoch
                output_dir = f'./Epoched_Data/{"Resting_Closed" if recording.experiment_id == "resting_closed" else "Resting_Open"}/'
                output_filename = f"{recording.subject_id}_{recording.experiment_id.replace('_', '-')}_{i + 1}"
                output_path = f'{output_dir}{output_filename}.csv'
                os.makedirs(output_dir, exist_ok=True)
                epoch.to_csv(output_path)